In [150]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_13836\4143411418.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [1]:
from selenium.webdriver.common.keys import Keys
# import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from cdp_backend.pipeline import ingestion_models
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

In [29]:
sessions = []

# Get Datetime:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)

video_link = driver.find_element(by='id', value='VideoIcon_0_0')
video_link.click()
video = WebDriverWait(driver, 4).until(
    ec.presence_of_element_located((By.CSS_SELECTOR, '.close span'))
    # .ytp-right-controls .yt-uix-sessionlink
)

link = WebDriverWait(driver, 2).until(
    ec.presence_of_element_located((By.CSS_SELECTOR, 'iframe'))
    # .ytp-right-controls .yt-uix-sessionlink
)
# video.get_attribute('src')
sessions.append(
    ingestion_models.Session(
        # Get Video Link:
        video_uri=link.get_attribute('src'),
        session_index=0,
        session_datetime=dt
    )
)
# close_link = video.find_element(By.CSS_SELECTOR, '.modal-header button')
video.text
# sessions

''

In [151]:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)
dt.isoformat()
dt.year, dt.month, dt.day

(2022, 6, 30)

In [152]:
# Inspecting each minute_item:
li = driver.find_elements(by='css selector', value='li.customLineHeight')
event_minutes_items = []
month_convert = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}
for list in li[:6]:
    try:
        if list.find_element(by='css selector', value='a').get_attribute('class') == 'hrefnowrap':
            link = list.find_element(by='css selector', value='.hrefnowrap')#.get_attribute('href')
            link.click()
            before = driver.window_handles[0]
            after = driver.window_handles[1]
            driver.switch_to.window(after)
            element = WebDriverWait(driver, 2).until(
                ec.presence_of_element_located((By.CSS_SELECTOR, 'body'))
            )
            # print(element.text)
            minutes_item = element.find_element(By.CSS_SELECTOR, '.togglec .ng-binding')
            name = minutes_item.text.replace('This link open a new window\n', '')
            title = element.find_element(By.CSS_SELECTOR, '.legisDetails-tog p').text
            m_type = element.find_element(By.CSS_SELECTOR, '.togglec h5').text.split(' ')[0]
            decision = element.find_element(By.CSS_SELECTOR, 'span.detail_Status').text
            
            tables = element.find_elements(By.CSS_SELECTOR, '.col_full.nobottommargin')
            for table in tables:
                if 'Legislative History' in table.text:
                    rows = table.find_elements(By.CSS_SELECTOR, 'table tr')
                    rows = rows[1:-1]
                    # print(rows[0].text)
                    for row in rows:
                        vote_date = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text #.split(' ')
                        vote_date = vote_date.replace(vote_date[:3], month_convert[vote_date[:3]])
                        vote_date = datetime.strptime(vote_date, "%B %d, %Y")
                        if (vote_date.year, vote_date.month, vote_date.day) == (dt.year, dt.month, dt.day):
                            vote_link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a')
                            vote_link.click()
                            element = WebDriverWait(driver, 2).until(
                                ec.presence_of_element_located((By.CSS_SELECTOR, '.vote-table'))
                            )
                            # print(element.text)
                            print('a')
                            break

            driver.switch_to.window(before)
        else:
            try:
                model = ingestion_models.EventMinutesItem(
                    minutes_item = ingestion_models.MinutesItem(
                        list.find_element(by='css selector', value='.custom-rcaview-action-review').text
                    ),
                    decision = list.find_element(by='css selector', value='.col-md-12 b').text.replace('Action Taken: ', '')
                )
                event_minutes_items.append(model)
            except (selenium.common.exceptions.NoSuchElementException):
                continue
    except (selenium.common.exceptions.NoSuchElementException):
        continue
event_minutes_items

a


[EventMinutesItem(minutes_item=MinutesItem(name='Adoption of the agenda.', description=None, external_source_id=None), index=None, matter=None, supporting_files=None, decision='Adopted as amended', votes=None),
 EventMinutesItem(minutes_item=MinutesItem(name='Acceptance of minutes\nRegular Meeting of June 16, 2022', description=None, external_source_id=None), index=None, matter=None, supporting_files=None, decision='Accepted', votes=None),
 EventMinutesItem(minutes_item=MinutesItem(name='Referral of petitions, communications, and reports to the proper Committees.', description=None, external_source_id=None), index=None, matter=None, supporting_files=None, decision='Referred', votes=None)]

In [147]:
driver.switch_to.window(before)

In [149]:
driver.quit()